In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [2]:
import math


In [3]:
# carregar base de dados
import os.path
fileName = os.path.join('C:\spark\Data', 'teste_validacao.csv')
numPartitions = 4
rawData = sc.textFile(fileName, numPartitions)


In [4]:
Outlook = rawData.map(lambda x: x.split(",")[0])
Temperature = rawData.map(lambda x: x.split(",")[1])
Humidity = rawData.map(lambda x: x.split(",")[2])
Windy = rawData.map(lambda x: x.split(",")[3])
Play = rawData.map(lambda x: x.split(",")[4])

In [42]:
def Entropia (classe):
    """Calcula a Entropia de Shannon de uma distribuição de dados.

    Args:
        classe (RDD): RDD contendo conjunto de dados a ser calculado a entropia.
                      Valores devem ser categóricos.

    Returns:
        float: valor de Entropia de Shannon calculado para o RDD.
    """
    #counts calcula paralelamente o conteúdo do RDD
    #como tuplas contendo (tipo, quantidade)
    counts = (classe.map(lambda x: (x, 1))
                    .reduceByKey(lambda a,b: a + b))
    # n recebe o valor total de itens do RDD
    n = classe.count()
    
    # probs calcula a probabilidade de cada um dos estados do RDD
    probs = counts.map(lambda x: x[1]/float(n))
    
    # Entropia calcula a entropia do RDD
    ## a função map faz o calculo da Entropia de cada um dos estados
    ## a função reduce faz o somatório da entropia de Shannon
    entropia = (probs.map(lambda p: -p*math.log(p,2))
                     .reduce(lambda a,b: a + b))
    
    # retorna valor escalar referênte a entropia do RDD.
    return entropia

 
def infoGain (feature, classe, H):
    """Calcula o ganho de informação de um atributo em relação a uma classe.

    Args:
        feature (RDD): RDD contendo os conjuntos de dados do atributo a ser
                       calculado o Ganho de Informação
        
        classe (RDD): RDD contendo conjunto de dados da classe
        
        H (float): Entropia da Classe, previamente calculada.

    Returns:
        float: valor de ganho de informação (redução da Entropia) que o atributo fornece sobre a classe
    """
    # calcula paralelamente o conteúdo do RDD
    # como tuplas contendo (tipo, quantidade)
    feat_count = feature.map(lambda x: (x, 1))\
                        .reduceByKey(lambda a,b: a + b)\
                        .collect()
    # calcula as Entropias de um conjunto da classe dado cada um dos estados do atributo   
    entropiasN = [Entropia(classe.zip(feature).filter(lambda x: x[1]==v).map(lambda x: x[0]))  for v,_ in feat_count]
    
    # calcula a quantidade de itens no atributo
    n = classe.count()
        
    # calcula o ganho de informação do atributo.
    ig = H - sum([(f[1]/float(n))*p for f,p in zip(feat_count, entropiasN)])
    
    return ig
    

In [18]:
H =  Entropia (Play)
print H


0.940285958671


In [43]:
igOutlook = infoGain(Outlook, Play, H)
print igOutlook

0.246749819774


In [44]:
igTemperature = infoGain(Temperature, Play, H)
print igTemperature



0.029222565659


In [45]:
igHumidity = infoGain(Humidity, Play, H)
print igHumidity

0.151835501362


In [46]:
igWindy = infoGain(Windy, Play, H)
print igWindy

0.0481270304083
